<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/assignment/ass6/bdm/Thruth%20Archive%201/Big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dealing with large datasets poses challenges in terms of memory efficiency and processing speed. In this exploration, we'll employ five smart strategies to effectively manage substantial datasets**

1. Load Less Data
2. Use Chunking
3. Optimize Data Types
4. Sampling
5. Parallelize with Dask

### [Dataset](https://www.kaggle.com/datasets/rmjacobsen/property-listings-for-5-south-american-countries?select=ar_properties.csv)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Assignment 2"

In [ ]:
!kaggle datasets download -d rmjacobsen/property-listings-for-5-south-american-countries

 97% 460M/475M [00:05<00:00, 118MB/s]
100% 475M/475M [00:05<00:00, 83.8MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  property-listings-for-5-south-american-countries.zip
replace ar_properties.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

ar_properties.csv  gdrive						 sample_data
co_properties.csv  pe_properties.csv					 uy_properties.csv
ec_properties.csv  property-listings-for-5-south-american-countries.zip


In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import psutil
import time
import os

Here, we called all the necessary libraries.

Traditional

In [ ]:
file_path='/content/ar_properties.csv'
file_size = os.path.getsize(file_path) / (1024 * 1024)  # Convert to megabytes

df = pd.read_csv(file_path)

In [ ]:
df.head(10)

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,2019-09-15,-26.815439,-65.312393,Argentina,Tucumán,Yerba Buena,...,NaN,NaN,NaN,NaN,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,2019-09-15,-26.839469,-65.212790,Argentina,Tucumán,NaN,...,1.0,55.0,41.0,NaN,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.424820,-64.181225,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.421242,-64.190798,Argentina,Córdoba,Córdoba,...,1.0,48.0,48.0,NaN,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.426064,-64.180042,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
5,vY1UjrDt8jFIGPtJfdpNZQ==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.429089,-64.189291,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,Departamento - Nueva Cordoba,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
6,D1KQkn+XsaOHRKVUuQRtpg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.418640,-64.188257,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Vélez Sarsfield al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
7,2yVnYhBoYRQ5r/mfQ32Mwg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.425732,-64.180824,Argentina,Córdoba,Córdoba,...,1.0,40.0,40.0,NaN,NaN,Mensual,Departamento - Nueva Cordoba,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
8,t0kNTwjTWOdat8Z87X7QLw==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.423450,-64.184201,Argentina,Córdoba,Córdoba,...,NaN,55.0,55.0,NaN,NaN,Mensual,1 DORMITORIO | Ituzaingó al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
9,fY4sOgwvFSS5Qz2M66X7/A==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.423824,-64.181516,Argentina,Córdoba,Córdoba,...,1.0,44.0,44.0,NaN,NaN,Mensual,1 DORMITORIO | Obispo Salguero al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal


In this example, we employed traditional data handling approaches without incorporating any advanced methods.

Data processing

In [ ]:
df.size

25000000

In [ ]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')

In [ ]:
df.columns = ['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude', 'Longitude',
              'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4', 'L5', 'L6',
              'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area', 'Covered_Area', 'Price', 'Currency',
              'Price_Period', 'Title', 'Description', 'Property_Type', 'Operation_Type']


In [ ]:
df.columns

Index(['ID', 'Ad_Type', 'Start_Date', 'End_Date', 'Created_On', 'Latitude',
       'Longitude', 'Country_L1', 'Province_L2', 'City_L3', 'Neighbourhood_L4',
       'L5', 'L6', 'Rooms', 'Bedrooms', 'Bathrooms', 'Total_Area',
       'Covered_Area', 'Price', 'Currency', 'Price_Period', 'Title',
       'Description', 'Property_Type', 'Operation_Type'],
      dtype='object')

In [ ]:
df.shape

(1000000, 25)

There are i million rows and 25 columns in this dataset.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 25 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   ID                1000000 non-null  object 
 1   Ad_Type           1000000 non-null  object 
 2   Start_Date        1000000 non-null  object 
 3   End_Date          1000000 non-null  object 
 4   Created_On        1000000 non-null  object 
 5   Latitude          850472 non-null   float64
 6   Longitude         849456 non-null   float64
 7   Country_L1        1000000 non-null  object 
 8   Province_L2       1000000 non-null  object 
 9   City_L3           944673 non-null   object 
 10  Neighbourhood_L4  249223 non-null   object 
 11  L5                4616 non-null     object 
 12  L6                0 non-null        float64
 13  Rooms             531101 non-null   float64
 14  Bedrooms          368640 non-null   float64
 15  Bathrooms         770378 non-null   float64
 16  T

In [ ]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
df = df.drop_duplicates()
result = df.count()

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

Execution Time: 7.043523788452148 seconds
Memory Usage Before: 1441.72265625 MB
Memory Usage After: 1656.55859375 MB
Memory Usage Difference: 214.8359375 MB


1. The operation took approximately 7.04 seconds to complete.
2. The memory usage before the operation was 1441.72 megabytes (MB).
3. The memory usage after the operation increased to 1656.56 megabytes (MB).
4. The difference in memory usage (after - before) is 214.84 megabytes (MB). This represents the additional memory consumed during the execution of the operation.

In [ ]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
null_counts = df.isnull().sum()

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

# Display the result of df.isnull().sum()
print("Null Counts:")
print(null_counts)

Execution Time: 2.353555917739868 seconds
Memory Usage Before: 1656.55859375 MB
Memory Usage After: 1669.94921875 MB
Memory Usage Difference: 13.390625 MB
Null Counts:
ID                        0
Ad_Type                   0
Start_Date                0
End_Date                  0
Created_On                0
Latitude             149528
Longitude            150544
Country_L1                0
Province_L2               0
City_L3               55327
Neighbourhood_L4     750777
L5                   995384
L6                  1000000
Rooms                468899
Bedrooms             631360
Bathrooms            229622
Total_Area           421223
Covered_Area         450055
Price                 48673
Currency              54505
Price_Period         581101
Title                    28
Description              25
Property_Type             0
Operation_Type            0
dtype: int64


In [ ]:
# l5 and L6 are not clear, so drop them
df = df.drop(['L5', 'L6'], axis=1)


In [ ]:
#Impute Categorical Values:
df['City_L3'].fillna(df['City_L3'].mode()[0], inplace=True)
df['Neighbourhood_L4'].fillna(df['Neighbourhood_L4'].mode()[0], inplace=True)
df['Price_Period'].fillna(df['Price_Period'].mode()[0], inplace=True)


In [ ]:
#text
df['Title'].fillna('', inplace=True)
df['Description'].fillna('', inplace=True)


In [ ]:
#Impute Numerical Values
df['Latitude'].fillna(df['Latitude'].mean(), inplace=True)
df['Longitude'].fillna(df['Longitude'].mean(), inplace=True)
df['Rooms'].fillna(df['Rooms'].mean(), inplace=True)
df['Bedrooms'].fillna(df['Bedrooms'].mean(), inplace=True)
df['Bathrooms'].fillna(df['Bathrooms'].mean(), inplace=True)
df['Total_Area'].fillna(df['Total_Area'].mean(), inplace=True)
df['Covered_Area'].fillna(df['Covered_Area'].mean(), inplace=True)
df['Price'].fillna(df['Price'].mean(), inplace=True)

In [ ]:
df.isnull().sum()

ID                      0
Ad_Type                 0
Start_Date              0
End_Date                0
Created_On              0
Latitude                0
Longitude               0
Country_L1              0
Province_L2             0
City_L3                 0
Neighbourhood_L4        0
Rooms                   0
Bedrooms                0
Bathrooms               0
Total_Area              0
Covered_Area            0
Price                   0
Currency            54505
Price_Period            0
Title                   0
Description             0
Property_Type           0
Operation_Type          0
dtype: int64

for now, lets keep Currency

In [ ]:
print(df['ID'].nunique())
print(df['Ad_Type'].nunique())
print(df['Start_Date'].nunique())
print(df['End_Date'].nunique())
print(df['Created_On'].nunique())
print(df['Latitude'].nunique())
print(df['Longitude'].nunique())
print(df['Country_L1'].nunique())
print(df['Province_L2'].nunique())
print(df['City_L3'].nunique())
print(df['Neighbourhood_L4'].nunique())
print(df['Rooms'].nunique())
print(df['Bedrooms'].nunique())
print(df['Bathrooms'].nunique())
print(df['Total_Area'].nunique())
print(df['Covered_Area'].nunique())
print(df['Price'].nunique())
print(df['Currency'].nunique())
print(df['Price_Period'].nunique())
print(df['Title'].nunique())
print(df['Description'].nunique())
print(df['Property_Type'].nunique())
print(df['Operation_Type'].nunique())


1000000
1
368
448
368
330930
333351
4
42
1267
982
36
79
21
5448
2801
20033
4
3
497587
685520
10
3


In [ ]:
df.describe()

,Latitude,Longitude,Rooms,Bedrooms,Bathrooms,Total_Area,Covered_Area,Price
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1.000000e+06
mean,-34.669838,-59.333241,2.881469,2.098199,1.682252,483.990998,4.372907e+02,2.857189e+05
std,2.745181,2.532343,1.234549,1.516532,0.935525,3085.644863,3.750450e+04,1.135263e+07
min,-54.975338,-180.000000,1.000000,-13.000000,1.000000,-200.000000,-1.700000e+02,0.000000e+00
25%,-34.720683,-59.333241,2.881469,2.098199,1.000000,78.000000,6.500000e+01,3.500000e+04
50%,-34.620537,-58.512473,2.881469,2.098199,1.682252,483.990998,2.900000e+02,1.050000e+05
75%,-34.521630,-58.389603,3.000000,2.098199,2.000000,483.990998,4.372907e+02,2.500000e+05
max,85.051129,-4.496101,40.000000,390.000000,20.000000,200000.000000,2.410000e+07,1.000000e+10


##New

Load Less Data

In this, we will strategically load less data to optimize data usage.
we will measure the time and memory usage to see if there's any differences.

In [ ]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
selected_columns = ['id', 'ad_type', 'start_date', 'end_date', 'bathrooms',
                    'surface_total', 'surface_covered', 'price', 'currency',
                    'price_period', 'title', 'description', 'property_type', 'operation_type']
df_selected = pd.read_csv(file_path, usecols=selected_columns)

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")

Execution Time: 18.588422298431396 seconds
Memory Usage Before: 2102.53125 MB
Memory Usage After: 3361.19140625 MB
Memory Usage Difference: 1258.66015625 MB


1. The operation took approximately 18.59 seconds to complete. This is the duration between measuring the time before and after performing the data loading operation.

2. The memory usage before the operation was 2102.53 megabytes (MB). This is the initial memory footprint of your Python process before loading the data.

3. The memory usage after the operation increased to 3361.19 megabytes (MB). This is the memory footprint of your Python process after loading the selected columns from the dataset.

4. The difference in memory usage (after - before) is 1258.66 megabytes (MB). This represents the additional memory consumed during the execution of the data loading operation

In [ ]:
df.head(5)

,ID,Ad_Type,Start_Date,End_Date,Created_On,Latitude,Longitude,Country_L1,Province_L2,City_L3,...,Bathrooms,Total_Area,Covered_Area,Price,Currency,Price_Period,Title,Description,Property_Type,Operation_Type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,2019-09-15,-26.815439,-65.312393,Argentina,Tucumán,Yerba Buena,...,1.682252,483.990998,437.290718,285718.897994,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,2019-09-15,-26.839469,-65.212790,Argentina,Tucumán,Mar del Plata,...,1.000000,55.000000,41.000000,285718.897994,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.424820,-64.181225,Argentina,Córdoba,Córdoba,...,1.000000,45.000000,45.000000,285718.897994,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.421242,-64.190798,Argentina,Córdoba,Córdoba,...,1.000000,48.000000,48.000000,285718.897994,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.426064,-64.180042,Argentina,Córdoba,Córdoba,...,1.000000,45.000000,45.000000,285718.897994,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal


##Optimize Data Types


We fine-tuned data types to maximize efficiency and minimize memory consumption.

In [ ]:
# Example: Optimize data types
df['Latitude'] = df['Latitude'].astype('float32')
df['Longitude'] = df['Longitude'].astype('float32')
df['Rooms'] = df['Rooms'].astype('float32')
df['Bedrooms'] = df['Bedrooms'].astype('float32')
df['Bathrooms'] = df['Bathrooms'].astype('float32')
df['Total_Area'] = df['Total_Area'].astype('float32')
df['Covered_Area'] = df['Covered_Area'].astype('float32')
df['Price'] = df['Price'].astype('float32')



##Sampling

In this part we will implement sampling methodologies to extract meaningful insights from a subset of the dataset.

In [ ]:
sample_size=100000
# Function to measure time and memory usage
def measure_time_and_memory_usage(code_block):
    # Measure memory usage before
    memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

    # Measure time before
    start_time = time.time()

    # Perform operations
    exec(code_block)

    # Measure time after
    end_time = time.time()

    # Measure memory usage after
    memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

    # Calculate differences
    execution_time = end_time - start_time
    memory_difference = memory_after - memory_before

    # Print results
    print(f"Execution Time: {execution_time} seconds")
    print(f"Memory Usage Before: {memory_before} MB")
    print(f"Memory Usage After: {memory_after} MB")
    print(f"Memory Usage Difference: {memory_difference} MB")

# Code block 1
code_block_1 = f"""
file_path = '{file_path}'
sample_size = {sample_size}
df_random_sample = pd.read_csv(file_path).sample(n=sample_size, random_state=42)
"""

measure_time_and_memory_usage(code_block_1)

# Code block 2
code_block_2 = f"""
file_path = '{file_path}'
sample_size = {sample_size}
df_stratified_sample = pd.read_csv(file_path)
stratified_sample = df_stratified_sample.groupby('property_type', group_keys=False).apply(lambda x: x.sample(min(len(x), sample_size), random_state=42))
"""

measure_time_and_memory_usage(code_block_2)

# Code block 3
code_block_3 = f"""
file_path = '{file_path}'
sample_size = {sample_size}
df_time_based_sample = pd.read_csv(file_path)
df_time_based_sample['start_date'] = pd.to_datetime(df_time_based_sample['start_date'])
df_time_based_sample = df_time_based_sample.sort_values(by='start_date').head(sample_size)
"""

measure_time_and_memory_usage(code_block_3)

# Code block 4
code_block_4 = f"""
file_path = '{file_path}'
df_percentage_sample = pd.read_csv(file_path).sample(frac=0.1, random_state=42)
"""

measure_time_and_memory_usage(code_block_4)

Execution Time: 20.22060251235962 seconds
Memory Usage Before: 3250.1796875 MB
Memory Usage After: 4217.140625 MB
Memory Usage Difference: 966.9609375 MB
Execution Time: 23.515020847320557 seconds
Memory Usage Before: 4021.03515625 MB
Memory Usage After: 4591.99609375 MB
Memory Usage Difference: 570.9609375 MB
Execution Time: 20.315021753311157 seconds
Memory Usage Before: 4177.58984375 MB
Memory Usage After: 4452.44140625 MB
Memory Usage Difference: 274.8515625 MB
Execution Time: 19.07874870300293 seconds
Memory Usage Before: 4127.59765625 MB
Memory Usage After: 4269.2890625 MB
Memory Usage Difference: 141.69140625 MB


1. Code Block 2 has a relatively higher memory usage difference compared to the others, indicating that the stratified sampling operation may have a larger impact on memory, considering the sample size of 100,000.

2. Code Block 3 shows a moderate memory usage difference, suggesting that time-based sampling has a moderate impact on memory, with a sample size of 100,000.

3. Code Block 4 has the lowest memory usage difference, indicating that percentage-based sampling has a relatively lower impact on memory, with a sample size of 10% of the total dataset.

In [ ]:

# Example: Random sampling
sample_df = df.sample(frac=0.1)  # Adjust the fraction as needed


In [ ]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 16822 to 265272
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                100000 non-null  object 
 1   Ad_Type           100000 non-null  object 
 2   Start_Date        100000 non-null  object 
 3   End_Date          100000 non-null  object 
 4   Created_On        100000 non-null  object 
 5   Latitude          100000 non-null  float32
 6   Longitude         100000 non-null  float32
 7   Country_L1        100000 non-null  object 
 8   Province_L2       100000 non-null  object 
 9   City_L3           100000 non-null  object 
 10  Neighbourhood_L4  100000 non-null  object 
 11  Rooms             100000 non-null  float32
 12  Bedrooms          100000 non-null  float32
 13  Bathrooms         100000 non-null  float32
 14  Total_Area        100000 non-null  float32
 15  Covered_Area      100000 non-null  float32
 16  Price           

The DataFrame, comprising 100,000 entries across 23 columns, occupies approximately 15.3 megabytes in memory. The data types include float32 for numerical values and object for categorical or text-based values. Most columns exhibit complete data, with the exception of the "Currency" column, which has 94,581 non-null entries.

##Parallelize with Dask

Now, we will try Dask. Dask is a powerful Python library designed for parallel computing and handling larger-than-memory datasets seamlessly. It extends the capabilities of Pandas, NumPy, and other popular libraries, enabling users to scale their data processing tasks from a single machine to distributed clusters effortlessly.

In [ ]:
import dask.dataframe as dd


In [ ]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
ddf = dd.read_csv(file_path, usecols=selected_columns)

# Handle errors during computation
try:
    result = ddf.groupby('price').mean().compute()
except pd.errors.ParserError as e:
    print(f"Error during computation: {e}")

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")


/usr/local/lib/python3.10/dist-packages/dask/dataframe/groupby.py:362: FutureWarning: The default value of numeric_only will be changed to False in the future when using dask with pandas 2.0
  warnings.warn(


Error during computation: Error tokenizing data. C error: EOF inside string starting at row 69686
Execution Time: 2.576700448989868 seconds
Memory Usage Before: 4127.56640625 MB
Memory Usage After: 4433.6328125 MB
Memory Usage Difference: 306.06640625 MB


1. The code took approximately 2.58 seconds to execute. This is the duration between measuring the time before and after performing the Dask operations.

2. The memory usage before the Dask operations was 4127.57 megabytes (MB).

3. The memory usage after the Dask operations increased to 4433.63 megabytes (MB).

4. The difference in memory usage (after - before) is 306.07 megabytes (MB). This represents the additional memory consumed during the execution of the Dask operations.

In [ ]:
if 'result' in locals():
    print("Computation Result:")
    print(result)

Computation Result:
ID                  1000000
Ad_Type             1000000
Start_Date          1000000
End_Date            1000000
Created_On          1000000
Latitude             850472
Longitude            849456
Country_L1          1000000
Province_L2         1000000
City_L3              944673
Neighbourhood_L4     249223
L5                     4616
L6                        0
Rooms                531101
Bedrooms             368640
Bathrooms            770378
Total_Area           578777
Covered_Area         549945
Price                951327
Currency             945495
Price_Period         418899
Title                999972
Description          999975
Property_Type       1000000
Operation_Type      1000000
dtype: int64


/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


##Use Chunking

In this last part we used chunking. Chunking is a strategy used to process large datasets by breaking them into smaller, more manageable pieces. This approach helps to avoid memory issues that might arise when trying to load or process an entire dataset at once.

In [ ]:
# Measure memory usage before
memory_before = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Measure time before
start_time = time.time()

# Perform operations
selected_columns = ['id', 'ad_type', 'start_date', 'end_date', 'lat', 'lon', 'bathrooms',
                    'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
                    'title', 'description', 'property_type', 'operation_type']
chunk_size = 10000

# Create an empty DataFrame to store the processed chunks
processed_chunks = []

# Iterate over chunks
for chunk in pd.read_csv(file_path, usecols=selected_columns, chunksize=chunk_size):
    # Remove duplicate rows
    chunk = chunk.drop_duplicates()

    # Remove empty rows (assuming empty rows have NaN values in all columns)
    chunk = chunk.dropna(how='all')

    # Append the processed chunk to the list
    processed_chunks.append(chunk)

# Concatenate the processed chunks into a final DataFrame
df_cleaned = pd.concat(processed_chunks, ignore_index=True)

# Measure time after
end_time = time.time()

# Measure memory usage after
memory_after = psutil.Process().memory_info().rss / (1024 * 1024)  # in megabytes

# Calculate differences
execution_time = end_time - start_time
memory_difference = memory_after - memory_before

# Print results
print(f"Execution Time: {execution_time} seconds")
print(f"Memory Usage Before: {memory_before} MB")
print(f"Memory Usage After: {memory_after} MB")
print(f"Memory Usage Difference: {memory_difference} MB")


Execution Time: 23.36314630508423 seconds
Memory Usage Before: 4405.74609375 MB
Memory Usage After: 5080.66015625 MB
Memory Usage Difference: 674.9140625 MB


1. The code took approximately 23.36 seconds to execute. This is the duration between measuring the time before and after performing the data cleaning operations on the chunks.

2. The memory usage before the chunk processing was 4405.75 megabytes (MB). This is the initial memory footprint of your Python process before cleaning the data.

3. The memory usage after the chunk processing increased to 5080.66 megabytes (MB). This is the memory footprint of your Python process after cleaning and concatenating the chunks.

4. The difference in memory usage (after - before) is 674.91 megabytes (MB). This represents the additional memory consumed during the execution of the chunk processing and concatenation operations.


In [ ]:
df_cleaned

,id,ad_type,start_date,end_date,lat,lon,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,-26.815439,-65.312393,NaN,NaN,NaN,NaN,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,-26.839469,-65.212790,1.0,55.0,41.0,NaN,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,-31.424820,-64.181225,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,-31.421242,-64.190798,1.0,48.0,48.0,NaN,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,-31.426064,-64.180042,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,ml17bb3zNa945e9dZWrJ6Q==,Propiedad,2020-02-20,2020-04-28,-38.011919,-57.535854,5.0,NaN,NaN,750000.0,USD,NaN,CHALET STELLA MARIS,Chalet en dos plantas sobre lote de 1100 m2 y ...,Otro,Venta
999996,O54TyNw4kMogEjaJqWhtLw==,Propiedad,2020-02-20,2020-04-28,-37.995254,-57.553228,5.0,NaN,NaN,180000.0,USD,NaN,Venta Dto 4 Amb con Dep Macrocentro,En av. Luro esquina Salta encontramos este am...,Otro,Venta
999997,Y1K0Rl31ib5DwJL5qqCiqg==,Propiedad,2020-02-20,2020-04-28,-37.983723,-57.544284,5.0,NaN,NaN,450000.0,USD,NaN,UNICA - HISTORICA - destino comercial,IDEAL DESARROLLO COMERCIAL - Pegado al Unzue -...,Otro,Venta
999998,xZ+W9ufh1Ugj2jPPqWwN8A==,Propiedad,2020-02-20,2020-04-28,-32.983590,-68.879191,5.0,NaN,NaN,500000.0,USD,NaN,CASA EN VENTA-CHACRAS DE CORIA,"Importante propiedad en calle Italia, Chacras ...",Otro,Venta
